In [ ]:
import pandas as pd
from IPython.display import Markdown
import plotly.graph_objects as go

import tb_incubator.constants as const
from tb_incubator.constants import set_project_base_path
from tb_incubator.model import build_model
from tb_incubator.plotting import display_plot, plot_tracked_outputs, plot_model_vs_actual
from tb_incubator.input import load_targets, load_param_info, load_report_section
from tb_incubator.utils import get_param_table
from tb_incubator.calibrate import get_all_priors

import arviz as az
az.style.use("arviz-doc")

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")
calib_out = project_paths["OUT_PATH"]

## Results

- [Calibration results](calibration_results.ipynb): Comparison between parameter prior and posterior distribution
- [Output ranges](spaghetti.ipynb): Output ranges
- [Scenarios](scenarios.ipynb): Scenario analysis

In [ ]:
targets = load_targets()
indicator_names = const.indicator_names
compartments = const.COMPARTMENTS
param_info = load_param_info()
fixed_params = param_info["value"]
covid_effects = {
    'detection_reduction':True,
    #'post_covid_improvement': True
}

In [ ]:
file_suffix = "p17_39"
idata = az.from_netcdf(calib_out / f'calib_full_out_{file_suffix}.nc')
likelihood_df = pd.read_hdf(calib_out / f'results_{file_suffix}.hdf', 'likelihood')
ldf_sorted = likelihood_df.sort_values(by="logposterior",ascending=False)
map_params = idata.posterior.to_dataframe().loc[ldf_sorted.index[0]].to_dict()
map_params_filtered = {k: v for k, v in map_params.items() 
                      if "_dispersion" not in k}
fixed_params.update(map_params_filtered)

In [ ]:
params = {}

In [ ]:
map_params_filtered

## Introduction

In [ ]:
#load_report_section('reporting.yaml', 'introduction')
#load_report_section('reporting.yaml', 'introduction2')

## Model construction

In [ ]:
model, desc = build_model(fixed_params, xpert_improvement=True, covid_effects=covid_effects)
#Markdown(desc)

In [ ]:
model.run(fixed_params | params)

## Results

In [ ]:
outs = model.get_derived_outputs_df()

In [ ]:
fig=plot_model_vs_actual(
    outs, targets['notification'], "notification", "Notification", "Modelled vs Data", "Actual data"
)
fig.update_xaxes(range=[1980, 2035])

In [ ]:
fig=plot_model_vs_actual(
    outs, targets['adults_prevalence_pulmonary_target'], "adults_prevalence_pulmonary", "adults_prevalence_pulmonary", "Modelled vs Data", "Actual data"
)
fig.update_xaxes(range=[2000, 2035])

In [ ]:
fig=plot_model_vs_actual(
    outs, targets['incidence'], "incidence", "incidence", "Modelled vs Data", "Actual data"
)
fig.update_xaxes(range=[2000, 2035])

In [ ]:
tracked_outputs = [
    "base_detection",
    "diagnostic_improvement",
    "covid_impacts",
    "final_detection",
    "notif_ratio"
]

plot_tracked_outputs(outs, tracked_outputs, layout=(3,2), plot_start_date=1990)

In [ ]:
#load_report_section('reporting.yaml', 'result-conclusion')

\newpage
## Supplementary

In [ ]:
#| label: tbl-params
#| tbl-cap: Model parameters
#| tbl-cap-location: top

prior_names = [p.name for p in get_all_priors()]
fixed_param_table = get_param_table(param_info, prior_names)
fixed_param_table
#Markdown(fixed_param_table.to_markdown())


\newpage
## References